### Zero-shot classification sur un texte français

#### Objectif
L'objectif de la classification zero-shot est de classer des textes selon des catégories définies à l'exécution et non pas à l'entraînement.

#### Méthode
La première étape consiste à charger un modèle de réseaux de neurones. En utilisant l'API de huggingface, si le modèle n'a pas déjà été téléchargé, il le sera automatiquement, donc assurez-vous d'avoir une connexion Internet.

Dans cette section, nous allons utiliser l'objet pipeline afin que le pré et post-traitement des réponses soient gérées automatiquement.
Les différents modèles compatibles peuvent être retrouvés sur le site de Huggingface [https://huggingface.co/models?pipeline_tag=zero-shot-classification](https://huggingface.co/models?pipeline_tag=zero-shot-classification)

Vous pouvez retrouver toutes les instructions d'utilisation de chaque modèle sur leur page dédiée. Par exemple, pour le modèle utilisé ci-dessous: [https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-mnli-xnli)

In [9]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")



Device set to use cpu


#### Données de test
Pour un premier test, les données à indiquer sont:
- Une chaîne de caractère à classifier. Avec le modèle proposé, le texte peut être proposé en 16 différentes langue dont l'anglais et le français
- Une liste de chaîne de caractère correspondant aux classes. Vous pouvez les modifier pour tester les aptitudes et les limites du modèles.

In [7]:
sequence_to_classify = "La classification zero-shot est une sujet de deep learning ne nécessite pas une connaissance préalable des classes utilisées."
candidate_labels = ["politique", "économie",  "littérature", "technologies"]

#sequence_to_classify = "Zero-shot classification is a deep learning classfication topic, where methods do not need to know the classes before inference. "
#candidate_labels = ["politics", "economics",  "litterature", "technology"]

#### Exécutions
Vous pouvez ensuite exécuter le pipeline pour en voir les résultats.

In [8]:
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

{'sequence': 'La classification zero-shot est une sujet de deep learning ne nécessite pas une connaissance préalable des classes utilisées.', 'labels': ['technologies', 'politique', 'littérature', 'économie'], 'scores': [0.34595373272895813, 0.28582847118377686, 0.23987437784671783, 0.12834341824054718]}


#### Résultats
Ce classifieur renvoie un dictionnaire comprenant ces informations:
- sequence: La chaîne de caractère donnée en entrée
- labels: Les différentes classes proposées. L'ordre est à retenir car c'est cet ordre auquel correspond les scores.
- scores: Les scores par classe. Les scores s'additionnent à 1, sauf si l'option multi_tabel est à true.

In [11]:
print("Resultats de la classification:")
print("Phrase:", output["sequence"])
print("Scores:")
n_classes = len(output["labels"])
for k in range(n_classes):
    print(output["labels"][k],"->",output["scores"][k])

Resultats de la classification:
Phrase: La classification zero-shot est une sujet de deep learning ne nécessite pas une connaissance préalable des classes utilisées.
Scores:
technologies -> 0.34595373272895813
politique -> 0.28582847118377686
littérature -> 0.23987437784671783
économie -> 0.12834341824054718


#### Évaluation via un dataset
Avec la libraire datasets, également liée à Huggingface, il est possible de télécharger et d'utiliser différents jeux de données. Pour les plus gros, dédiés à l'entraînement ou au fine-tuning de réseaux de neurones, les élements peuvent être séparés en catégorie: "training", "validation" et "test". 

Dans le cas du jeu de données dans ce notebook, il s'agit uniquement d'un jeu de test, il n'y a donc que la sous-partie "test".

In [12]:
from datasets import load_dataset
ds = load_dataset("AdaptLLM/law_knowledge_prob")

data = ds['test']
print(len(data))
print(data[0])


10000
{'label': 'Employment', 'text': "Executive agrees to be employed with the Company, and the Company agrees to employ Executive, during the Term and on the terms and conditions set forth in this Agreement. Executive agrees during the term of this Agreement to devote substantially all of Executive’s business time, efforts, skills and abilities to the performance of Executive’s duties to the Company and to the furtherance of the Company's business.", 'options': ['Employment', 'Indemnifications', 'Waivers', 'Intellectual Property']}


Vous remarquez que chaque élément de ce dataset contient plusieurs termes:
- label: La classification attendue
- text: La phrase à classifier
- options: Les classes possible. La classification attendue, ainsi que des classifications fausses en font partie.

#### Evaluation


La première étape consiste à créer les prédictions de notre réseau. Dans la section ci-dessous nous utilisons une quantité limitée de données pour éviter des temps de calcul trop longs (10 éléments = 10-15 secondes)

In [13]:
n = 10
references = ds['test']['label'][:n]
predictions = []

for k in range(n):
    results = classifier(ds['test']['text'][k], ds['test']['options'][k])
    predictions.append(results['labels'][0])
     

Une fois les prédictions et resultats récupérés, on peut les comparer afin d'avoir la proportion de bonnes réponses.

Pour évaluer la qualité des résultats, plusieurs métriques existent selon le cas d'application. Pour cet exemple en classification zero-shot, la métrique d'évaluation que nous allons utiliser est la proporition de données classées correctement.

In [ ]:
import evaluate
metric = evaluate.load('exact_match')
exact_match = metric.compute(references=references,predictions=predictions)
print(exact_match)
print(exact_match['exact_match'])

NameError: name 'exac' is not defined